In [1]:
import numpy as np

In [2]:
def read_annFile(filepath):
    with open(filepath) as f:
        lines = [line.rstrip() for line in f]
    ann0 = []
    ann0_boxes = []
    ann0_racks = []
    for ll in lines:
        line = ll.split(", ")
        a = line[:]
        a = np.array(a)
        if a[0] == "Box":
            ann0_boxes.append(a[1:].astype(np.float))
        else:
            ann0_racks.append(a[1:].astype(np.float))
        ann0.append(a[:])
    ann0 = np.array(ann0)
    ann0_boxes = np.array(ann0_boxes)
    ann0_racks = np.array(ann0_racks)
    return ann0, ann0_boxes, ann0_racks

def findCorrespondances(box0, box1):
    margin_yz = 2
    shape0 = box0.shape
    shape1 = box1.shape
    p0, p1 = 0, 0
    cor0 = []
    cor1 = []
    corres_indices = []
    while p0 < shape0[0] and p1 < shape1[0]:
        if np.linalg.norm(box0[p0, -2:] - box1[p1, -2:]) < margin_yz:
            cor0.append(box0[p0])
            cor1.append(box1[p1])
            corres_indices.append((p0, p1))
            p0+=1 
            p1+=1
        elif np.linalg.norm(box0[p0, -2:] - box1[p1, -2:]) < margin_yz:
            cor0.append(box0[p0])
            cor1.append(box1[p1 + 1])
            corres_indices.append((p0, p1+1))
            p0+=1 
            p1+=2
        elif np.linalg.norm(box0[p0, -2:] - box1[p1, -2:]) < margin_yz:    
            cor0.append(box0[p0 + 1])
            cor1.append(box1[p1])
            corres_indices.append((p0+1, p1))
            p0+=2 
            p1+=1
    return np.array(cor0), np.array(cor1), corres_indices

# Pass correspondances
def findTranlation(cor0, cor1):
    n0, n1 = cor0.shape
    margin_x = 2
    corr1 = []
    corr0 = []
    avg_translation = np.array([0., 0., 0.])
    pt = 0
    for i in range(n0):
        if np.linalg.norm(cor0[i, 3] - cor1[i, 3]) < margin_x:
            avg_translation += (cor0[i, :3] - cor1[i, :3])
            pt+=1
    avg_translation = avg_translation / pt
    return avg_translation

def findMergedDis(filepath_0, filepath_1):
    ann_0, boxes_0, racks_0 = read_annFile(filepath_0)
    ann_1, boxes_1, racks_1 = read_annFile(filepath_1)
    print(ann_0, "\n\n", ann_1, "\n")
    print(boxes_0, "\n\n", boxes_1, "\n")
    print(racks_0, "\n\n", racks_1)
    
    corBox_0, corBox_1, corBox_indices = findCorrespondances(boxes_0, boxes_1)
    corRack_0, corRack_1, corRack_indices = findCorrespondances(racks_0, racks_1)
    
    avg_trans = findTranlation(corBox_0, corBox_1)

    # Translate the boxes and racks
    ind1 , ind2 = 0, 0
    print("Corresponding indices for boxes: ", corBox_indices)
    print("Corresponding indices for racks: ", corRack_indices)
    return avg_trans

def findMergedLayout(path_ann):
    pass
# avg tanslations between two frames for all the frames 

In [3]:
filepath_0 = "ann_17.txt"
filepath_1 = "ann_18.txt"

print("\n", findMergedDis(filepath_0, filepath_1))

[['Box' '80' '255' '270' '5' '20' '69']
 ['Box' '158' '255' '289' '16' '20' '31']
 ['Box' '189' '255' '298' '17' '19' '14']
 ['Box' '221' '256' '292' '16' '24' '24']
 ['Box' '322' '255' '292' '16' '19' '25']
 ['Box' '352' '255' '269' '16' '18' '70']
 ['Box' '384' '255' '289' '17' '20' '31']
 ['Shelf' '83' '255' '256' '14' '49' '97']
 ['Shelf' '183' '255' '256' '161' '49' '97']
 ['Shelf' '355' '255' '256' '161' '49' '97']] 

 [['Box' '79' '255' '270' '17' '20' '69']
 ['Box' '164' '255' '289' '17' '20' '31']
 ['Box' '194' '255' '298' '17' '19' '14']
 ['Box' '226' '256' '292' '16' '24' '24']
 ['Box' '328' '255' '292' '16' '19' '25']
 ['Box' '358' '255' '269' '16' '18' '70']
 ['Box' '389' '255' '289' '17' '20' '31']
 ['Shelf' '82' '255' '256' '25' '49' '97']
 ['Shelf' '188' '255' '256' '161' '49' '97']
 ['Shelf' '360' '255' '256' '161' '49' '97']] 

[[ 80. 255. 270.   5.  20.  69.]
 [158. 255. 289.  16.  20.  31.]
 [189. 255. 298.  17.  19.  14.]
 [221. 256. 292.  16.  24.  24.]
 [322. 255